In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.recipe_db
collection = db.low_calorie

In [108]:
url = 'https://www.allrecipes.com/recipes/84/healthy-recipes/'

response = requests.get(url)

soup = bs(response.text, 'html.parser')

In [109]:
soup.prettify()

'<!DOCTYPE html>\n<!--\n        /((((((\\\\\\\\\n=======((((((((((\\\\\\\\\\\n     ((           \\\\\\\\\\\\\\\n     ( (*    _/      \\\\\\\\\\\\\\\n       \\    /  \\      \\\\\\\\\\\\________________\n        |  |   |       </                  ((\\\\\\\\\n        o_|   /        /                      \\ \\\\\\\\    \\\\\\\\\\\\\\\n             |  ._    (                        \\ \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\n             | /                       /       /    \\\\\\\\\\\\\\     \\\\\n     .______/\\/     /                 /       /         \\\\\\\n    / __.____/    _/         ________(       /\\\n   / / / ________/`_________\'         \\     /  \\_\n  / /  \\ \\                             \\   \\ \\_  \\\n ( <    \\ \\                             >  /    \\ \\\n  \\/     \\\\_                           / /       > )\n          \\_|                         / /       / /\n                                    _//       _//\n                                   /_|       /_|\n-->\n<ht

In [110]:
results = soup.find_all('a', class_='carouselNav__link recipeCarousel__link')
results

[<a class="carouselNav__link recipeCarousel__link" data-tracking-content-headline="healthy appetizers" href="https://www.allrecipes.com/recipes/1319/healthy-recipes/appetizers/" tabindex="0">
 <div alt="Healthy Appetizers" aria-label="" class="component lazy-image lazy-image-udf aspect_1x1 align-default" data-alt="Healthy Appetizers" data-content-type="" data-crop-percentage="100" data-high-density="false" data-main-recipe="" data-orientation="default" data-original-height="1277" data-original-width="1278" data-shop-image="false" data-special-crop="" data-src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2F2020%2F07%2F22%2F6583475.jpg" data-title="Healthy Appetizers" data-tracking-zone="image" data-width="      100
                                    ">
 <noscript>
 <img alt="Healthy Appetizers" src="https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2

In [111]:
health_cats = ['low-calorie recipes','low-carb recipes','low-cholesterol recipes','low-fat recipes','weight-loss recipes','healthy snacks']

In [112]:
cat =  [x['href'] for x in results if x['data-tracking-content-headline'] in health_cats]
cat

['https://www.allrecipes.com/recipes/12155/healthy-recipes/snacks/',
 'https://www.allrecipes.com/recipes/1232/healthy-recipes/low-calorie/',
 'https://www.allrecipes.com/recipes/742/healthy-recipes/low-carb/',
 'https://www.allrecipes.com/recipes/737/healthy-recipes/low-cholesterol/',
 'https://www.allrecipes.com/recipes/1231/healthy-recipes/low-fat/',
 'https://www.allrecipes.com/recipes/22607/healthy-recipes/weight-loss/']

In [119]:
main_cats={}

for c in cat:
    r = requests.get(c)
    soup = bs(r.text, 'html.parser')
    title = soup.find('h1', class_= 'category-page-heading').text
    results = soup.find_all('a', class_= 'card__titleLink manual-link-behavior')
    links = [y['href'] for y in results]
    
    main_cats[title] = list(set(links))
    
main_cats

{'Healthy Snack Recipes': ['https://www.allrecipes.com/recipe/223467/granola-bars/',
  'https://www.allrecipes.com/recipe/15006/spiced-sweet-roasted-red-pepper-hummus/',
  'https://www.allrecipes.com/recipe/219456/healthier-buffalo-chicken-dip/',
  'https://www.allrecipes.com/recipe/19677/great-garlic-knots/',
  'https://www.allrecipes.com/recipe/236557/salt-and-vinegar-potato-chips/',
  'https://www.allrecipes.com/recipe/20482/baked-tortilla-chips/',
  'https://www.allrecipes.com/recipe/24720/texas-caviar-i/',
  'https://www.allrecipes.com/recipe/54165/balsamic-bruschetta/',
  'https://www.allrecipes.com/recipe/214722/pickled-daikon-radish-and-carrot/',
  'https://www.allrecipes.com/recipe/260520/apple-frog-for-kids/',
  'https://www.allrecipes.com/recipe/26692/annies-fruit-salsa-and-cinnamon-chips/',
  'https://www.allrecipes.com/recipe/236025/salt-and-pepper-skillet-fries/',
  'https://www.allrecipes.com/recipe/239017/fresh-tomato-salsa-restaurant-style/',
  'https://www.allrecipes.

In [127]:
for k, v in main_cats.items():
    print(k,v[0])
    

Healthy Snack Recipes https://www.allrecipes.com/recipe/223467/granola-bars/
Low-Calorie Recipes https://www.allrecipes.com/recipe/11887/pesto-pasta/
Low-Carb Recipes https://www.allrecipes.com/recipe/244478/grilled-shrimp-over-zucchini-noodles/
Low-Cholesterol Recipes https://www.allrecipes.com/recipe/24944/snow-ice-cream-ii/
Low-Fat Recipes https://www.allrecipes.com/recipe/256599/tomato-cucumber-kachumbar/
Weight-Loss Recipes https://www.allrecipes.com/recipe/86230/szechwan-shrimp/


In [114]:
main_cats

[['https://www.allrecipes.com/recipe/260520/apple-frog-for-kids/',
  'https://www.allrecipes.com/recipe/260520/apple-frog-for-kids/',
  'https://www.allrecipes.com/recipe/255799/black-bean-hummus-without-tahini/',
  'https://www.allrecipes.com/recipe/255799/black-bean-hummus-without-tahini/',
  'https://www.allrecipes.com/recipe/254364/baked-sweet-potato-coins/',
  'https://www.allrecipes.com/recipe/254364/baked-sweet-potato-coins/',
  'https://www.allrecipes.com/recipe/246372/perfectly-dry-roasted-chickpeas/',
  'https://www.allrecipes.com/recipe/246372/perfectly-dry-roasted-chickpeas/',
  'https://www.allrecipes.com/recipe/240551/fruit-skewers-with-apple-cinnamon-dipping-sauce/',
  'https://www.allrecipes.com/recipe/240551/fruit-skewers-with-apple-cinnamon-dipping-sauce/',
  'https://www.allrecipes.com/recipe/239017/fresh-tomato-salsa-restaurant-style/',
  'https://www.allrecipes.com/recipe/239017/fresh-tomato-salsa-restaurant-style/',
  'https://www.allrecipes.com/recipe/176957/bake

In [152]:
u = 'https://www.allrecipes.com/recipe/260520/apple-frog-for-kids/'

resp = requests.get(u)
soup = bs(resp.text, 'html.parser')
title = soup.find('h1', class_= 'headline heading-content').text
rating = soup.find('span', class_='review-star-text').text.replace('\n',"").strip()
num_ratings = soup.find('span', class_='ugc-ratings-item').text.replace('\n',"").strip()

time = soup.find_all('div', class_= 'recipe-meta-item')
time_dict = {}
time_dict['prep'] = time[0].find('div', class_= 'recipe-meta-item-body').text.replace('\n',"").strip()
time_dict['total'] = time[1].find('div', class_= 'recipe-meta-item-body').text.replace('\n',"").strip()
ingredients

{'prep': '5 mins', 'total': '5 mins'}